# P(s) for rectangular off-diagonal regions

In [ ]:
import pprint
import os
import sys
import collections

import bioframe
import click
import cooler
import cooltools
import cooltools.expected
import matplotlib.pyplot as plt
import matplotlib.gridspec
from matplotlib.lines import Line2D
import numpy as np
from numpy.lib.function_base import average
from pairlib.scalings import norm_scaling
import pandas as pd
import pairlib
import pairlib.scalings
import pairtools
from diskcache import Cache
from itertools import combinations
import itertools

from pandas.io.pytables import IndexCol

from diskcache import Cache
from itertools import combinations

from pandas.io.pytables import IndexCol

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
cooler_paths = {
    'wt': '',
    'top2-td': ''
    }
resolution = 2000
out_path = ''
assembly = 'sacCer3'
region_size = 40000
exclude_chroms = []
centromeres_path = ''
title = 'cdc20-td cdc20: 40kbp region around CENs'

In [ ]:
coolers = {label: cooler.Cooler('::/resolutions/'.join((path, str(resolution)))) for label, path in cooler_paths.items()}

In [ ]:
chromsizes = bioframe.fetch_chromsizes(assembly, filter_chroms=False, as_bed=True)
chromsizes = chromsizes[~chromsizes.chrom.isin(exclude_chroms)]
#chromsizes.set_index('chrom', inplace=True)
cens = pd.read_csv(centromeres_path, delim_whitespace=True, header=None, names=['chrom', 'start', 'end'])

cens = bioframe.ops.expand(cens, region_size // 2)

In [ ]:
from cooltools.lib.numutils import persistent_log_bins
from scipy.stats import binned_statistic

fig, ax = plt.subplots(figsize=(10, 10))

for label, clr in coolers.items():
    scalings = []
    for idx, cen in cens.iterrows():
        end = chromsizes[chromsizes['chrom'] == cen.chrom]['end'].values[0]
        region1 = (cen.chrom, 0, end)
        region2 = (cen.chrom, cen.start if cen.start >= 0 else 0, cen.end if cen.end <= end else end)
        mat = clr.matrix(balance=False).fetch(region1, region2)
        scalings.append(mat.mean(axis=1))

    # average chromosome scalings
    scalings = [np.nanmean(i) for i in list(itertools.zip_longest(*scalings, fillvalue=np.nan))]
    # bincount
    
    # plot
    img = ax.loglog(
        scalings,
        label=label,
        lw=1,
        alpha=0.5
    )

ax.grid(lw=0.5, color='gray')
ax.set_xlim(1e1)
ax.set_xlabel('distance from CEN')
ax.set_ylabel('contact frequency')
ax.set_aspect(1.0)
ax.set_title(title)
plt.legend()
plt.savefig(out_path, bbox_inches='tight', dpi=300)